As data is a mix of categorical and continuous attributes, 2 models are used: GaussianNB model for continuous and CategoricalNB for categorical. The 2 classifiers are then combined.

In [1]:
import pandas as pd
import numpy as np
import settings
from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
datapath = "../data/"

fullTrainData = pd.read_excel(settings.labelledDatapath)

print(fullTrainData.columns)
print("Class balance:")
fullTrainData["class"].value_counts()

/home/peter/.local/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Index(['RowID', 'age', 'workclass', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'class'],
      dtype='object')
Class balance:


<=50K    24720
>50K      7841
Name: class, dtype: int64

In [3]:
#removing redundant columns
fullTrainData = fullTrainData.drop(labels=settings.redundantFeatures, axis=1)
fullTrainData.head()

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
#making labels numerical - 1 for >50K, 0 for <=50K or unknown
fullTrainData["class"] = fullTrainData["class"].map(lambda x: 1 if(x == ">50K") else 0)

#### Up until now, the start process was the same as for Decision Trees.

In [5]:
#encoding categorical features
fullTrainData[settings.categoricalFeatures] = fullTrainData[settings.categoricalFeatures].apply(LabelEncoder().fit_transform)
fullTrainData.head()

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,6,13,4,0,1,4,1,2174,0,40,38,0
1,50,5,13,2,3,0,4,1,0,0,13,38,0
2,38,3,9,0,5,1,4,1,0,0,40,38,0
3,53,3,7,2,5,0,2,1,0,0,40,38,0
4,28,3,13,2,9,5,2,0,0,0,40,4,0


In [6]:
#splitting features and labels
attributes = fullTrainData.drop(labels=["class"], axis=1)
labels = fullTrainData["class"]

#getting test and train set
trainFeatures, testFeatures, trainLabels, testLabels = train_test_split(attributes, labels, test_size=0.2, random_state=5) #random state is like a seed to allow repeatable results


In [7]:
categoricalModel = CategoricalNB()
gaussianModel = GaussianNB()

categoricalModel.fit(trainFeatures[settings.categoricalFeatures], trainLabels)
gaussianModel.fit(trainFeatures[settings.numericalFeatures], trainLabels)

GaussianNB()

#### Combining results
What we want:
$$P(C|\text{all features}) = \frac{P(\text{all features}|C)P(C)}{P(\text{all features})}$$

How we get it:

Because "Naive" (assumes independence)
$$P(\text{all features}) = P(\text{cat. features}).P(\text{num. features})$$
and
$$P(\text{all features}|C) = P(\text{cat. features}|C).P(\text{num. features}|C)$$

$$P(\text{cat. features}|C) = \frac{P(C|\text{cat. features})P(\text{cat. features})}{P(C)}$$
$$P(\text{num. features}|C) = \frac{P(C|\text{num. features})P(\text{num. features})}{P(C)}$$

so:

$$P(C|\text{all features}) = \frac{\frac{P(C|\text{cat. features})P(\text{cat. features})}{P(C)}.\frac{P(C|\text{num. features})P(\text{num. features})}{P(C)}.P(C)}{P(\text{all features})}$$
$$= \frac{\frac{P(C|\text{cat. features})P(\text{cat. features})P(C|\text{num. features})P(\text{num. features})}{P(C)}}{P(\text{all features})}$$
$$= \frac{P(C|\text{cat. features})P(C|\text{num. features})}{P(C)}$$


## Combining classifiers (Currently without logs - maybe try logs?)

In [12]:
catProb = categoricalModel.predict_proba(testFeatures[settings.categoricalFeatures])
print(catProb)
numProb = gaussianModel.predict_proba(testFeatures[settings.numericalFeatures])
print(numProb)
print(numProb.shape)

[[0.27724949 0.72275051]
 [0.99188163 0.00811837]
 [0.65537723 0.34462277]
 ...
 [0.14074946 0.85925054]
 [0.84384956 0.15615044]
 [0.9987202  0.0012798 ]]
[[1.40497799e-04 9.99859502e-01]
 [9.94530907e-01 5.46909341e-03]
 [9.98121651e-01 1.87834865e-03]
 ...
 [9.91675381e-01 8.32461863e-03]
 [9.77351971e-01 2.26480286e-02]
 [9.90186080e-01 9.81392003e-03]]
(6513, 2)


In [21]:
print(gaussianModel.class_prior_)
overallPreNorm = (catProb * numProb) / gaussianModel.class_prior_
print(overallPreNorm)
print(overallPreNorm.shape)

[0.75840756 0.24159244]
[[5.13614922e-05 2.99119023e+00]
 [1.30069503e+00 1.83781041e-04]
 [8.62525962e-01 2.67939550e-03]
 ...
 [1.84040595e-01 2.96074367e-02]
 [1.08746020e+00 1.46382876e-02]
 [1.30394118e+00 5.19875886e-05]]
(6513, 2)


In [25]:
totSum = np.sum(overallPreNorm, 1)
print(totSum.shape)
overall = overallPreNorm / totSum[:,None]           #final division step for normalising the probabilites back so their sum is 1
print(overall)

(6513,)
[[1.71706265e-05 9.99982829e-01]
 [9.99858725e-01 1.41274528e-04]
 [9.96903168e-01 3.09683184e-03]
 ...
 [8.61419567e-01 1.38580433e-01]
 [9.86717804e-01 1.32821956e-02]
 [9.99960132e-01 3.98679915e-05]]
